# Generate training data

### CISCO and Spaninsh Ens

In [5]:
import pandas as pd
import re

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
CISCO_PATH = "Schemes/Cisco.csv"         # Assicurati che il percorso sia corretto
ENS_PATH = "Schemes/SpanishENS.csv"      # Assicurati che il percorso sia corretto
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step1_cisco_ens.csv"

# ==============================================================================
# 2. CARICAMENTO DATI
# ==============================================================================
print("1. Caricamento file...")

# Caricamento Cisco
df_cisco = pd.read_csv(CISCO_PATH)
print(f"   - Cisco loaded: {df_cisco.shape} righe")

# Caricamento Spanish ENS (con separatore punto e virgola)
# Usiamo dtype=str per evitare problemi con codici interpretati come numeri
df_ens = pd.read_csv(ENS_PATH, sep=';', dtype=str)
print(f"   - Spanish ENS loaded: {df_ens.shape} righe")

# ==============================================================================
# 3. CREAZIONE LOOKUP TABLE (ENS INGLESE AGGREGATO)
# ==============================================================================
print("\n2. Indicizzazione e Aggregazione ENS (English)...")

ens_lookup = {}

# Raggruppiamo per codice perché nel CSV lo stesso codice può apparire più volte
# (es. una riga per il titolo, una riga per le domande)
grouped_ens = df_ens.groupby('Codice')

for code, group in grouped_ens:
    # Normalizza la chiave (codice)
    if pd.isna(code): continue
    clean_code = str(code).strip().lower()
    
    # Raccogli tutti i testi INGLESI disponibili per questo codice
    texts = []
    
    for _, row in group.iterrows():
        # Titolo Inglese
        if 'Titolo_EN' in row and not pd.isna(row['Titolo_EN']):
            t = str(row['Titolo_EN']).strip()
            if t: texts.append(t)
            
        # Domande Inglese
        if 'Domande_EN' in row and not pd.isna(row['Domande_EN']):
            q = str(row['Domande_EN']).strip()
            # Rimuovi i pipe '|' che separano le domande e sostituisci con spazio
            q = q.replace('|', ' ')
            if q: texts.append(q)
            
    # Unisci tutto il testo raccolto per questo controllo
    full_text_en = ". ".join(texts)
    
    # Pulizia finale (spazi doppi, ecc)
    full_text_en = re.sub(r'\s+', ' ', full_text_en).strip()
    
    if full_text_en:
        ens_lookup[clean_code] = full_text_en

print(f"   - Indicizzati {len(ens_lookup)} codici ENS univoci con testo in Inglese.")

# ==============================================================================
# 4. MAPPING E GENERAZIONE DATASET
# ==============================================================================
print("\n3. Generazione coppie di training (Cisco -> ENS)...")

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# Colonne Cisco dove cercare i riferimenti ENS
ens_columns = ['Spanish ENS BASIC Control', 'Spanish ENS Medium Control', 'Spanish ENS High Control']

for idx, row in df_cisco.iterrows():
    # Anchor (Cisco)
    # Combiniamo Titolo e Wording per la massima ricchezza semantica
    if pd.isna(row['Control Reference']): continue
    
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    anchor_text = f"{title}: {wording}".strip()
    
    # Target (ENS)
    # Cerchiamo i codici nelle 3 colonne
    target_codes = []
    for col in ens_columns:
        if col in row and not pd.isna(row[col]):
            # Split per virgola in caso ci siano più controlli in una cella
            codes_in_cell = str(row[col]).split(',')
            target_codes.extend([c.strip().lower() for c in codes_in_cell])
    
    # Rimuovi duplicati e itera
    unique_targets = set(target_codes)
    
    for t_code in unique_targets:
        if not t_code: continue
        
        if t_code in ens_lookup:
            # MATCH TROVATO
            positive_text = ens_lookup[t_code]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_code,
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'Spanish ENS (EN)'
            })
            stats["matches"] += 1
        else:
            stats["missing"].add(t_code)

# ==============================================================================
# 5. SALVATAGGIO
# ==============================================================================
df_train = pd.DataFrame(training_data)
df_train.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO FINALE")
print("-" * 30)
print(f"Totale coppie generate: {len(df_train)}")
print(f"Codici mancanti (nel file ENS): {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:5]}")
print(f"\nFile salvato come: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_train[['anchor_text', 'target_text']].head(2))

1. Caricamento file...
   - Cisco loaded: (713, 32) righe
   - Spanish ENS loaded: (213, 8) righe

2. Indicizzazione e Aggregazione ENS (English)...
   - Indicizzati 169 codici ENS univoci con testo in Inglese.

3. Generazione coppie di training (Cisco -> ENS)...
------------------------------
RISULTATO FINALE
------------------------------
Totale coppie generate: 318
Codici mancanti (nel file ENS): 28
Esempio mancanti: ['mp.s.1', 'mp.com.9', 'op.cont.1', 'mp.info.6', 'mp.info.5']

File salvato come: TrainAndTestData/trainingData/training_data_step1_cisco_ens.csv

Anteprima dati:
                                         anchor_text  \
0  Control Self-Assessments: Independent Control ...   
1  Control Self-Assessments: Independent Control ...   

                                         target_text  
0  Metrics system. Taking into account the securi...  
1  Certified components. Is the Catalog of Inform...  


### CISCO and Spaninsh Ens

In [7]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
# Percorsi basati sulla tua struttura di cartelle
PATH_CISCO = "Schemes/Cisco.csv"
PATH_SECNUM = "Schemes/Secnumcloud.csv"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step2_cisco_secNumCloud.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE SECNUMCLOUD
# ==============================================================================
print("1. Caricamento e indicizzazione SecNumCloud...")

try:
    # Tentativo di lettura standard (virgola)
    df_sec = pd.read_csv(PATH_SECNUM)
    
    # Verifica colonne critiche
    if 'ID' not in df_sec.columns or 'Description_EN' not in df_sec.columns:
        # Fallback: prova con punto e virgola se la prima lettura non ha parsato le colonne
        df_sec = pd.read_csv(PATH_SECNUM, sep=';')

    print(f"   - SecNumCloud caricato: {len(df_sec)} righe.")
except Exception as e:
    print(f"   - ERRORE critico nel caricamento di SecNumCloud: {e}")
    exit()

# Creazione dizionario di lookup: ID (lowercase) -> Descrizione Inglese
secnum_lookup = {}
for _, row in df_sec.iterrows():
    if pd.isna(row['ID']): continue
    
    # Normalizzazione ID (es: "5.1.A " -> "5.1.a")
    clean_id = str(row['ID']).strip().lower()
    
    # Estrazione testo inglese
    if 'Description_EN' in row and not pd.isna(row['Description_EN']):
        desc = str(row['Description_EN']).strip()
        secnum_lookup[clean_id] = desc

print(f"   - Indicizzati {len(secnum_lookup)} controlli SecNumCloud univoci (EN).")

# ==============================================================================
# 3. MAPPING CISCO -> SECNUMCLOUD
# ==============================================================================
print("\n2. Generazione coppie di training (Cisco -> SecNumCloud)...")

df_cisco = pd.read_csv(PATH_CISCO)
print(f"   - Cisco caricato: {len(df_cisco)} righe.")

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# La colonna Cisco che contiene i riferimenti
target_col = 'SecNumCloud Control'

for idx, row in df_cisco.iterrows():
    # Verifica se c'è un mapping
    if target_col not in row or pd.isna(row[target_col]):
        continue
    
    # 1. Preparazione Anchor (Cisco)
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    # Uniamo titolo e wording per dare contesto completo al modello
    anchor_text = f"{title}: {wording}".strip()
    
    # 2. Preparazione Target (SecNumCloud IDs)
    # Cisco può contenere liste tipo "5.1.a, 5.2.b" o usare newline
    raw_refs = str(row[target_col]).replace('\n', ',')
    target_ids_list = [t.strip().lower() for t in raw_refs.split(',')]
    
    # 3. Matching
    for t_id in set(target_ids_list): # set() per evitare duplicati sulla stessa riga
        if not t_id: continue
        
        if t_id in secnum_lookup:
            # MATCH TROVATO
            positive_text = secnum_lookup[t_id]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_id,      # ID normalizzato
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'SecNumCloud (EN)'
            })
            stats["matches"] += 1
        else:
            # MATCH NON TROVATO (ID presente in Cisco ma non nel file SecNumCloud)
            stats["missing"].add(t_id)

# ==============================================================================
# 4. SALVATAGGIO FILE STEP 2
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 2")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
print(f"Codici SecNumCloud non trovati: {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:3]}...")
print(f"\nFile salvato: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_result[['anchor_id', 'target_id', 'target_text']].head(2))

1. Caricamento e indicizzazione SecNumCloud...
   - SecNumCloud caricato: 287 righe.
   - Indicizzati 261 controlli SecNumCloud univoci (EN).

2. Generazione coppie di training (Cisco -> SecNumCloud)...
   - Cisco caricato: 713 righe.
------------------------------
RISULTATO STEP 2
------------------------------
Nuove coppie generate: 567
Codici SecNumCloud non trovati: 33
Esempio mancanti: ['8.5.b', '9.6.k', '12.5.d']...

File salvato: TrainAndTestData/trainingData/training_data_step2_cisco_secNumCloud.csv

Anteprima dati:
  anchor_id target_id                                        target_text
0     CCF 1  18.2.1.a  The service provider must document and impleme...
1     CCF 1    18.4.a  The service provider must document and impleme...
